In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from epoch_based_evolution import SearchSpace
import load_data

# Load Data

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data.load_openml_dataset(dataset_id=334, scaling=True, random_seed=None, return_as='tensor')
input_size, output_size = load_data.get_tensor_sizes(X_train, y_train)

Class column is not numeric. Applying LabelEncoder.
Data loaded successfully! as <class 'torch.Tensor'>
Training data shape: torch.Size([384, 6])


In [7]:
search_space = SearchSpace(
    input_size=input_size,
    output_size=output_size,          
    # min_layers=2,           # Minimum number of hidden layers
    # max_layers=5,           # Maximum number of hidden layers
    # min_neurons=16,         # Minimum neurons per layer
    # max_neurons=256,        # Maximum neurons per layer
    # activation_fns=[nn.ReLU, nn.LeakyReLU],  # Activation functions to sample
    # dropout_rates=[0, 0.1, 0.2],             # Dropout rates to sample
    # min_learning_rate=0.0001,                # Minimum learning rate
    # max_learning_rate=0.01,                  # Maximum learning rate
    # random_seeds=[42, 13, 2024],             # Random seeds for reproducibility
    # min_batch_size=32,                       # Minimum batch size
    # max_batch_size=512                       # Maximum batch size
)


In [342]:
architecture = search_space.sample_architecture()
batch_size = architecture['batch_size']
model = search_space.create_model(architecture)
print(model)

DynamicNN(
  (network): Sequential(
    (0): Linear(in_features=6, out_features=448, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=448, out_features=237, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=237, out_features=486, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=486, out_features=479, bias=True)
    (7): Sigmoid()
    (8): Linear(in_features=479, out_features=42, bias=True)
    (9): Sigmoid()
    (10): Linear(in_features=42, out_features=2, bias=True)
  )
  (criterion): CrossEntropyLoss()
)


In [340]:
# Create DataLoaders
train_dataset, train_loader = load_data.create_dataset_and_loader(X_train, y_train,
                                                        batch_size=batch_size)
val_dataset, val_loader = load_data.create_dataset_and_loader(X_val, y_val, 
                                                    batch_size=batch_size)
test_dataset, test_loader = load_data.create_dataset_and_loader(X_test, y_test,
                                                      batch_size=batch_size)

In [445]:
search_space.train_model(model, train_loader, val_loader, num_epochs=1)

(0.6615638136863708, 0.625)